# Hyperparameter tuning
We outline the various challenges we faced when designing the experiments meant to tune our hyperparameters. If this doesn't interest you, skip to **"Final Experimental Setup"**.
## Initial Experimental Setup
The goal is to test various different models across various datasets and parameters. Initially, the goal was this:
1. Three datasets: numerical, complete, FAMD complete
    - Numerical: Just the numerical and ordinal data. This is roughly 30 columns.
    - Complete: Both the numerical data and the categorical data, which is one hot encoded. This is roughly 500 columns.
    - FAMD complete: This is the complete data, but the categorical columns have been prepared for PCA.
2. Yes/no PCA: We want to test the effect of PCA on improving model performance.
3. 2 models: Linear, Polynomial
4. Hyperparameter set 1: `alpha`
   - This is just different values of regularization for the model.
5. Hyperparameter set 2: `degree`
   - This is strictly for polynomial regression, where we try different degrees for polynomial features.

Suppose the number of alphas being picked is $n$ and the number of degrees being picked is $m$. In total, that's $3 \text{ datasets} \times 2 \text{ yes/no pca} \times (4\text{ linear reg alphas} + 1 \times (n\text{ polynomial reg alphas} \times m\text{ polynomial reg degrees}) = 6 \times (4 + n \times m)$. If we have $4$ choices of $n$ and $m$, then it'll be $6 \times (4 + 16) = 120$ different models we are evaluating. Note that we're not simply just multiplying each layer with each other because linear regression will not be tuning different degrees, which would result in $192$ models.
### Caveats to the initial setup
Upon working with the experiment, we came across several roadblocks and realizations.
1. It doesn't make sense to use the FAMD complete dataset on anything other than PCA. The whole point of FAMD is to make a dataset prepared for PCA. Otherwise, without PCA, it's functionally the same as the complete dataset. **Practically, we should only test the FAMD complete dataset in tandem with PCA.**
2. The degrees of the polynomial features preprocessing step is too computationally expensive for the complete datasets. Let's say that we are working with $32 \text{ GB}$ of memory on our machine.

Assuming that we have $1800$ rows with $500$ columns, and each column is a `float` costing 8 bytes, then the space cost of `degree=3` for the complete dataset is:
$$
\begin{aligned}
\sum_{i=1}^3{500 \choose i}&=20833750 \text{ samples} \\
\implies 20833750 \times 8 \text{ bytes} \times \frac{1\text{ GB}}{10^9 \text{ bytes}} &\approx 300 \text{ GB}
\end{aligned}
$$

For `degree=2`, the cost is approximately $1.8 \text{ GB}$. 

Meanwhile, if we have $1800$ rows with only $30$ columns, it will take `degree=7` to cross $32 \text{ GB}$
$$
\begin{aligned}
\sum_{i=1}^7{30 \choose i}&\approx2.8\times 10^6 \text{ samples} \\
\implies 2.8 \times 10^6 \times 8 \text{ bytes} \times \frac{1\text{ GB}}{10^9 \text{ bytes}} &\approx 40 \text{ GB}
\end{aligned}
$$

For `degree=6`, the cost is approximately $11 \text{ GB}$.

Upon further research, however, the columns use `float64` around $120$ bytes per cell of data, further limiting the actual degrees we can use; not even examining other factors that limit our memory allowance; just the size of the data.

**Through experiments, we've determined which degrees we can use. Practically, if we use the whole dataset, we can only go up to approximately `degree=2`, while if we use strictly the numerical dataset, we can go up to approximately `degree=3`, thus limiting our combinations for experimentation.**
### Outline of final experimental setup
The datasets are the only real limitation for what we can experiment with. Given what we know now, our final experimental setup should look like this, again assuming 4 alphas and 4 degrees.
- The numerical dataset can be tested with everything.
    - $2 \times (4 \times 4 + 4)=40$ models based on the numerical dataset. 
- The complete dataset can be tested with everything except polynomial degree, which should be limited to 2.
    - $2 \times (4 \times 1 + 4) = 16$ models based on the complete dataset.
- The FAMD complete dataset can be tested with everything except: No PCA, and polynomial degree, which should be limited to 2.
    - $1 \times (4 \times 1 + 4) = 8$ models based on the FAMD complete dataset.

So in total, $64$ models are being trained, assuming we keep the same 4-4 split of alphas and degrees.
## Final Experimental Setup
Numerical features will always be scaled.

The numerical dataset will be trained, generating $30$ models:
- With and without PCA.
- Linear Regression, $5$ choices of `alpha`.
- Polynomial Regression, $5$ choices of `alpha` crossed with $2$ choices of degrees.

The whole dataset will be trained, generating $20$ models:
- With and without PCA.
- Linear Regression, $5$ choices of `alpha`.
- Polynomial Regression, $5$ choices of `alpha` with only `degree=2`.

The FAMD complete dataset will be trained, generating $10$ models:
- With PCA
- Linear Regression, $5$ choices of `alpha`.
- Polynomial Regression, $5$ choices of `alpha` with only `degree=2`.

The result should be $60$ models.

# Experiment

## Read and create datasets
Here we create the datasets we'll be experimenting with, and split the target column from the feature matrices.

In [1]:
from data_processor import DataProcessor
import pandas as pd

# read data
df = pd.read_csv('../data/train.csv')
dp = DataProcessor(df)

# complete data
whole_df = dp.complete_data()
target = whole_df['SalePrice']
whole_df.drop('SalePrice', axis=1, inplace=True)
display(whole_df.head())

# numerical data
num_df = dp.numerical_data()
num_df.drop('SalePrice', axis=1, inplace=True)
display(num_df.head())

# FAMD complete data
_, famd_cat_df = dp.famd_data()
famd_df = pd.concat([num_df, famd_cat_df], axis=1)
display(famd_df.head())
display(target.head())

,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,65.0,8450,7,5,196.0,706,0,150,856,856,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,80.0,9600,6,8,0.0,978,0,284,1262,1262,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,68.0,11250,7,5,162.0,486,0,434,920,920,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,60.0,9550,7,5,0.0,216,0,540,756,961,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,84.0,14260,8,5,350.0,655,0,490,1145,1145,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,BsmtFinType1,BsmtFinType2,HeatingQC,Electrical,KitchenQual,Functional,FireplaceQu,GarageQual,GarageCond,PoolQC
0,65.0,8450,7,5,196.0,706,0,150,856,856,...,2.0,5.0,0.0,4.0,2.0,6.0,-1.0,4.0,4.0,-1.0
1,80.0,9600,6,8,0.0,978,0,284,1262,1262,...,0.0,5.0,0.0,4.0,3.0,6.0,4.0,4.0,4.0,-1.0
2,68.0,11250,7,5,162.0,486,0,434,920,920,...,2.0,5.0,0.0,4.0,2.0,6.0,4.0,4.0,4.0,-1.0
3,60.0,9550,7,5,0.0,216,0,540,756,961,...,0.0,5.0,2.0,4.0,2.0,6.0,2.0,4.0,4.0,-1.0
4,84.0,14260,8,5,350.0,655,0,490,1145,1145,...,2.0,5.0,0.0,4.0,2.0,6.0,4.0,4.0,4.0,-1.0


,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,65.0,8450,7,5,196.0,706,0,150,856,856,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.057354,0.0,0.0
1,80.0,9600,6,8,0.0,978,0,284,1262,1262,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.055132,0.000000,0.0,0.0
2,68.0,11250,7,5,162.0,486,0,434,920,920,...,0.0,0.125988,0.0,0.0,0.000000,0.000000,0.000000,0.057354,0.0,0.0
3,60.0,9550,7,5,0.0,216,0,540,756,961,...,0.0,0.000000,0.0,0.0,0.000000,0.056433,0.000000,0.000000,0.0,0.0
4,84.0,14260,8,5,350.0,655,0,490,1145,1145,...,0.0,0.000000,0.0,0.0,0.130189,0.000000,0.000000,0.057354,0.0,0.0


0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [2]:
import sys
sys.getsizeof(num_df['LotFrontage'].dtype)

120

## Define model pipeline
This is a function that creates the pipeline that `GridSearch` will be using.
- We always scale the numerical features -- we use `ColumnTransformer` to scale *only* the numerical features, as one hot encoded columns don't need scaling, and when the FAMD complete dataset is used, we don't want to scale the categorical columns.
- Generate the polynomial features if `degree>1`. This means if `degree=1`, this step will be skipped, and the model will just be automatically linear regression.
- Use `PCA`, if enabled, with an explained variance ratio of 95%
- Add the model. Will usually just be `Ridge`, which is `LinearRegression` that supports regularization

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

def create_pipeline(model, use_pca, degree=1):
    steps = []

    # always scale numerical data
    scaler = ColumnTransformer(
        transformers=[('scaler', StandardScaler(), num_df.columns)],
        remainder='passthrough'  # Keep one-hot-encoded categorical features
    )
    steps.append(('scaler', scaler))

    # generate polynomial combinations of the features if the model is polynomial
    if degree > 1:
        steps.append(('poly', PolynomialFeatures(degree=degree, include_bias=False)))

    # use pca, with an explained variance ratio of 95%
    if use_pca:
        steps.append(('pca', PCA(n_components=0.95)))

    # add the model
    steps.append(('model', model))

    return Pipeline(steps)

## Define experiment
This function uses the pipeline created to use grid search on the parameters put in. The results of the cross validation are trimmed.

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

def run_experiment(X, y, dataset_name, use_pca, degree, param_grid, verbose=1):
    # train and test
    pipeline = create_pipeline(Ridge(), use_pca, degree=degree)
    regr = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=10, scoring='neg_root_mean_squared_error', n_jobs=-1)
    regr.fit(X, y)

    # examine results
    cv_results = pd.DataFrame(regr.cv_results_)
    if verbose == 1:
        display(cv_results)

    # add to results list
    # 1. prune unnecessary stats
    pruned_results = cv_results[['mean_fit_time', 'params', 'mean_test_score']].copy()
    # 2. add degrees, dataset, and pca usage as parameters
    pruned_results['dataset_name'] = dataset_name
    pruned_results['use_pca'] = use_pca
    pruned_results['degree'] = degree
    # 3. add rmse and nrmse, in pretty format
    pruned_results['rmse'] = pruned_results['mean_test_score'].apply(lambda x: round(-x))  
    pruned_results['nrmse'] = pruned_results['mean_test_score'].apply(lambda x: f'{round(-x / y.mean() * 100, 1)}%')
    
    return pruned_results

## Experiment 1: Numerical dataset

In [5]:
# init params
param_grid = {
    'model__alpha' : [0.001, 0.01, 0.1, 1, 10]
}
degrees = [1, 2, 3]
pca_options = [True, False]
results = pd.DataFrame()

# run experiment
for use_pca in pca_options:
    for degree in degrees:
        exp_result = run_experiment(
            X=num_df, y=target, dataset_name='numerical', 
            use_pca=use_pca, degree=degree, param_grid=param_grid
        )
        results = pd.concat([results, exp_result], ignore_index=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018591,0.003996,0.005385,0.001162,0.001,{'model__alpha': 0.001},-25375.364675,-28142.530153,-25185.455554,-41487.017487,-38292.886814,-28977.232406,-27707.671185,-26927.254983,-62616.433441,-29874.775740,-33458.662244,10994.122885,5
1,0.014577,0.004147,0.004274,0.001587,0.010,{'model__alpha': 0.01},-25375.335285,-28142.527248,-25185.454821,-41486.999736,-38292.921339,-28977.236212,-27707.654967,-26927.281647,-62616.362843,-29874.706840,-33458.648094,10994.108095,4
2,0.014935,0.004551,0.005028,0.001374,0.100,{'model__alpha': 0.1},-25375.041457,-28142.498285,-25185.447549,-41486.822278,-38293.266610,-28977.274348,-27707.492873,-26927.548308,-62615.656980,-29874.017984,-33458.506667,10993.960209,3
3,0.015021,0.001098,0.005606,0.000352,1.000,{'model__alpha': 1},-25372.110620,-28142.216775,-25185.379781,-41485.052761,-38296.721786,-28977.662945,-27705.880506,-26930.216482,-62608.609428,-29867.143637,-33457.099472,10992.482595,2
4,0.016136,0.002308,0.005455,0.000932,10.000,{'model__alpha': 10},-25343.534145,-28140.197483,-25185.188240,-41467.854614,-38331.514268,-28982.258386,-27690.598318,-26957.049206,-62539.221675,-29799.797761,-33443.721410,10977.829074,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,4.654365,1.391234,0.023912,0.009523,0.001,{'model__alpha': 0.001},-25925.003722,-30989.573410,-31132.089543,-68976.443155,-44967.621394,-34805.212647,-26723.804786,-29075.012562,-55725.005522,-32268.422551,-38058.818929,13469.277776,5
1,4.962826,1.296776,0.022758,0.004110,0.010,{'model__alpha': 0.01},-25924.985869,-30989.575017,-31132.079391,-68976.413378,-44967.602845,-34805.210732,-26723.805171,-29075.009686,-55725.063793,-32268.418105,-38058.816399,13469.280076,4
2,3.556941,0.175736,0.018139,0.003073,0.100,{'model__alpha': 0.1},-25924.807355,-30989.591090,-31131.977888,-68976.115625,-44967.417371,-34805.191579,-26723.809038,-29074.980931,-55725.646499,-32268.373650,-38058.791103,13469.303069,3
3,3.298698,0.384135,0.009264,0.005893,1.000,{'model__alpha': 1},-25923.023634,-30989.752680,-31130.964787,-68973.139282,-44965.563796,-34805.000638,-26723.848620,-29074.694236,-55731.473190,-32267.929687,-38058.539055,13469.533042,2
4,1.934672,0.400906,0.006852,0.001413,10.000,{'model__alpha': 10},-25905.327524,-30991.453538,-31121.025917,-68943.494865,-44947.144246,-34803.148891,-26724.335715,-29071.912392,-55789.703555,-32263.548380,-38056.109502,13471.837125,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,45.848327,4.912154,0.061100,0.026550,0.001,{'model__alpha': 0.001},-68354.347196,-82079.343842,-113897.706760,-82669.642815,-95832.046580,-77250.834480,-72081.765196,-70549.808193,-89959.867179,-71939.435010,-82461.479725,13453.346913,5
1,48.037124,5.155532,0.068687,0.021755,0.010,{'model__alpha': 0.01},-68354.347197,-82079.343841,-113897.706645,-82669.642815,-95832.046581,-77250.834480,-72081.765196,-70549.808193,-89959.867179,-71939.435010,-82461.479714,13453.346887,4
2,40.602526,7.924776,0.042882,0.017388,0.100,{'model__alpha': 0.1},-68354.347200,-82079.343821,-113897.705497,-82669.642816,-95832.046583,-77250.834478,-72081.765198,-70549.808200,-89959.867179,-71939.435009,-82461.479598,13453.346618,3
3,42.821453,3.953056,0.023231,0.013518,1.000,{'model__alpha': 1},-68354.347232,-82079.343628,-113897.694015,-82669.642826,-95832.046609,-77250.834464,-72081.765219,-70549.808263,-89959.867179,-71939.435000,-82461.478444,13453.343929,2
4,26.576638,10.072417,0.015307,0.005508,10.000,{'model__alpha': 10},-68354.347551,-82079.341697,-113897.579197,-82669.642929,-95832.046864,-77250.834317,-72081.765430,-70549.808900,-89959.867186,-71939.434912,-82461.466898,13453.317037,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005724,0.000768,0.002398,0.000843,0.001,{'model__alpha': 0.001},-25906.129058,-34773.990799,-24592.281251,-41914.260612,-37645.883756,-29503.344771,-27709.409295,-26967.482944,-71662.876656,-30654.542329,-35133.020147,13253.821050,5
1,0.006531,0.001938,0.002475,0.000709,0.010,{'model__alpha': 0.01},-25906.033754,-34771.911713,-24592.192767,-41914.186378,-37645.928821,-29503.119904,-27709.470076,-26967.509241,-71660.182320,-30654.327675,-35132.486265,13253.106598,4
2,0.006655,0.001600,0.002736,0.000647,0.100,{'model__alpha': 0.1},-25905.081639,-34751.166303,-24591.309971,-41913.444714,-37646.379755,-29500.876571,-27710.076698,-26967.772181,-71633.329803,-30652.183081,-35127.162072,13245.988134,3
3,0.005992,0.001540,0.001987,0.000320,1.000,{'model__alpha': 1},-25895.651490,-34548.172448,-24582.682995,-41906.094766,-37650.916440,-29478.968064,-27716.025016,-26970.398435,-71373.578395,-30630.930130,-35075.341818,13177.318034,2
4,0.005553,0.000957,0.001878,0.000268,10.000,{'model__alpha': 10},-25809.208503,-32892.182497,-24513.428456,-41838.522616,-37698.362282,-29304.118567,-27765.087296,-26996.202509,-69423.792935,-30435.793566,-34667.669923,12674.782623,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.229739,0.045577,0.010198,0.001692,0.001,{'model__alpha': 0.001},-55389.728372,-253879.894060,-884735.395859,-103175.760671,-64959.799878,-61535.758026,-66502.645733,-136961.641983,-207489.089920,-63069.251677,-189769.896618,240627.474656,5
1,0.290090,0.070715,0.013948,0.006687,0.010,{'model__alpha': 0.01},-54965.673439,-177169.298776,-860707.512230,-99450.878923,-59729.213623,-52204.194410,-60941.657197,-100133.620474,-159325.387906,-56578.012145,-168120.544912,234753.496990,4
2,0.249097,0.056165,0.010770,0.002104,0.100,{'model__alpha': 0.1},-54545.940915,-75087.117348,-720916.360110,-91129.914340,-54191.661716,-43236.542010,-49060.888488,-51411.525021,-75390.867926,-49877.577636,-126484.839551,198666.065101,3
3,0.276434,0.044564,0.008515,0.002689,1.000,{'model__alpha': 1},-48476.172565,-49582.316339,-429626.846963,-85030.180636,-48508.007592,-40014.931769,-40107.802837,-42007.362375,-78514.943037,-48631.507177,-91050.007129,113830.471468,2
4,0.199519,0.029678,0.004839,0.000926,10.000,{'model__alpha': 10},-36295.674895,-35623.825055,-145462.768753,-74976.284261,-39352.105970,-35239.653832,-34299.035463,-33786.752794,-69811.433298,-43507.535927,-54835.507025,33425.883327,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,4.777974,0.638144,0.038555,0.007161,0.001,{'model__alpha': 0.001},-226618.743728,-593617.763029,-7.615862e+06,-349394.650796,-588513.479181,-374401.905056,-240756.579341,-64966.150003,-512655.554838,-329397.944092,-1.089619e+06,2.181158e+06,5
1,4.230888,0.502038,0.035362,0.007352,0.010,{'model__alpha': 0.01},-85957.329873,-572982.728000,-1.022915e+06,-166694.048509,-181644.943070,-96042.182414,-94851.342960,-64893.625203,-158504.486289,-122473.499706,-2.566959e+05,2.909402e+05,2
2,3.700241,0.105151,0.029946,0.004360,0.100,{'model__alpha': 0.1},-81671.379121,-464707.318047,-1.873199e+06,-150273.417889,-109542.939116,-80150.070991,-98172.998409,-63682.381425,-127558.022237,-106507.977093,-3.155465e+05,5.308591e+05,4
3,3.922904,0.419535,0.017411,0.005503,1.000,{'model__alpha': 1},-69719.604930,-277542.516204,-1.799009e+06,-130632.435044,-68844.154965,-72284.765061,-79382.160900,-56249.447496,-86831.029470,-97923.251648,-2.738418e+05,5.120600e+05,3
4,2.646178,1.040971,0.012718,0.003428,10.000,{'model__alpha': 10},-50979.185074,-136783.847301,-1.322189e+06,-125204.658831,-52641.906618,-58931.511566,-61623.349073,-45776.645547,-69817.917845,-74245.902331,-1.998194e+05,3.752886e+05,1


In [6]:
ranked_results = results.sort_values(by='mean_test_score', ascending=[False])
ranked_results

,mean_fit_time,params,mean_test_score,dataset_name,use_pca,degree,rmse,nrmse
4,0.016136,{'model__alpha': 10},-3.344372e+04,numerical,True,1,33444,18.5%
3,0.015021,{'model__alpha': 1},-3.345710e+04,numerical,True,1,33457,18.5%
2,0.014935,{'model__alpha': 0.1},-3.345851e+04,numerical,True,1,33459,18.5%
1,0.014577,{'model__alpha': 0.01},-3.345865e+04,numerical,True,1,33459,18.5%
0,0.018591,{'model__alpha': 0.001},-3.345866e+04,numerical,True,1,33459,18.5%
19,0.005553,{'model__alpha': 10},-3.466767e+04,numerical,False,1,34668,19.2%
18,0.005992,{'model__alpha': 1},-3.507534e+04,numerical,False,1,35075,19.4%
17,0.006655,{'model__alpha': 0.1},-3.512716e+04,numerical,False,1,35127,19.4%
16,0.006531,{'model__alpha': 0.01},-3.513249e+04,numerical,False,1,35132,19.4%
15,0.005724,{'model__alpha': 0.001},-3.513302e+04,numerical,False,1,35133,19.4%


## Experiment 2: Complete dataset

In [7]:
# same parameters, but degree locked to 2
degrees = [1, 2]
# run experiment - polynomial regression
for use_pca in pca_options:
    for degree in degrees:
        exp_result = run_experiment(
            X=num_df, y=target, dataset_name='complete', 
            use_pca=use_pca, degree=degree, param_grid=param_grid
        )    
        results = pd.concat([results, exp_result], ignore_index=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009592,0.002095,0.003448,0.000746,0.001,{'model__alpha': 0.001},-25375.364675,-28142.530153,-25185.455554,-41487.017487,-38292.886814,-28977.232406,-27707.671185,-26927.254983,-62616.433441,-29874.775740,-33458.662244,10994.122885,5
1,0.009641,0.001908,0.003226,0.000623,0.010,{'model__alpha': 0.01},-25375.335285,-28142.527248,-25185.454821,-41486.999736,-38292.921339,-28977.236212,-27707.654967,-26927.281647,-62616.362843,-29874.706840,-33458.648094,10994.108095,4
2,0.008601,0.001274,0.002871,0.000501,0.100,{'model__alpha': 0.1},-25375.041457,-28142.498285,-25185.447549,-41486.822278,-38293.266610,-28977.274348,-27707.492873,-26927.548308,-62615.656980,-29874.017984,-33458.506667,10993.960209,3
3,0.009672,0.001955,0.003332,0.000770,1.000,{'model__alpha': 1},-25372.110620,-28142.216775,-25185.379781,-41485.052761,-38296.721786,-28977.662945,-27705.880506,-26930.216482,-62608.609428,-29867.143637,-33457.099472,10992.482595,2
4,0.009297,0.002042,0.003141,0.000596,10.000,{'model__alpha': 10},-25343.534145,-28140.197483,-25185.188240,-41467.854614,-38331.514268,-28982.258386,-27690.598318,-26957.049206,-62539.221675,-29799.797761,-33443.721410,10977.829074,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,4.950474,1.323783,0.026579,0.006172,0.001,{'model__alpha': 0.001},-25925.003722,-30989.573410,-31132.089543,-68976.443155,-44967.621394,-34805.212647,-26723.804786,-29075.012562,-55725.005522,-32268.422551,-38058.818929,13469.277776,5
1,4.221107,1.515265,0.023811,0.003250,0.010,{'model__alpha': 0.01},-25924.985869,-30989.575017,-31132.079391,-68976.413378,-44967.602845,-34805.210732,-26723.805171,-29075.009686,-55725.063793,-32268.418105,-38058.816399,13469.280076,4
2,3.711720,1.032675,0.023421,0.004954,0.100,{'model__alpha': 0.1},-25924.807355,-30989.591090,-31131.977888,-68976.115625,-44967.417371,-34805.191579,-26723.809038,-29074.980931,-55725.646499,-32268.373650,-38058.791103,13469.303069,3
3,3.190864,0.727269,0.010111,0.005039,1.000,{'model__alpha': 1},-25923.023634,-30989.752680,-31130.964787,-68973.139282,-44965.563796,-34805.000638,-26723.848620,-29074.694236,-55731.473190,-32267.929687,-38058.539055,13469.533042,2
4,2.086864,0.489054,0.005969,0.002095,10.000,{'model__alpha': 10},-25905.327524,-30991.453538,-31121.025917,-68943.494865,-44947.144246,-34803.148891,-26724.335715,-29071.912392,-55789.703555,-32263.548380,-38056.109502,13471.837125,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015314,0.009315,0.005081,0.004875,0.001,{'model__alpha': 0.001},-25906.129058,-34773.990799,-24592.281251,-41914.260612,-37645.883756,-29503.344771,-27709.409295,-26967.482944,-71662.876656,-30654.542329,-35133.020147,13253.821050,5
1,0.006560,0.001427,0.002531,0.000642,0.010,{'model__alpha': 0.01},-25906.033754,-34771.911713,-24592.192767,-41914.186378,-37645.928821,-29503.119904,-27709.470076,-26967.509241,-71660.182320,-30654.327675,-35132.486265,13253.106598,4
2,0.007003,0.001316,0.002832,0.000645,0.100,{'model__alpha': 0.1},-25905.081639,-34751.166303,-24591.309971,-41913.444714,-37646.379755,-29500.876571,-27710.076698,-26967.772181,-71633.329803,-30652.183081,-35127.162072,13245.988134,3
3,0.007278,0.001366,0.002679,0.000642,1.000,{'model__alpha': 1},-25895.651490,-34548.172448,-24582.682995,-41906.094766,-37650.916440,-29478.968064,-27716.025016,-26970.398435,-71373.578395,-30630.930130,-35075.341818,13177.318034,2
4,0.006862,0.001308,0.002561,0.000497,10.000,{'model__alpha': 10},-25809.208503,-32892.182497,-24513.428456,-41838.522616,-37698.362282,-29304.118567,-27765.087296,-26996.202509,-69423.792935,-30435.793566,-34667.669923,12674.782623,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.214488,0.051669,0.009708,0.002016,0.001,{'model__alpha': 0.001},-55389.728372,-253879.894060,-884735.395859,-103175.760671,-64959.799878,-61535.758026,-66502.645733,-136961.641983,-207489.089920,-63069.251677,-189769.896618,240627.474656,5
1,0.282506,0.049186,0.013347,0.003529,0.010,{'model__alpha': 0.01},-54965.673439,-177169.298776,-860707.512230,-99450.878923,-59729.213623,-52204.194410,-60941.657197,-100133.620474,-159325.387906,-56578.012145,-168120.544912,234753.496990,4
2,0.267768,0.045015,0.009490,0.002067,0.100,{'model__alpha': 0.1},-54545.940915,-75087.117348,-720916.360110,-91129.914340,-54191.661716,-43236.542010,-49060.888488,-51411.525021,-75390.867926,-49877.577636,-126484.839551,198666.065101,3
3,0.294317,0.060275,0.008240,0.002851,1.000,{'model__alpha': 1},-48476.172565,-49582.316339,-429626.846963,-85030.180636,-48508.007592,-40014.931769,-40107.802837,-42007.362375,-78514.943037,-48631.507177,-91050.007129,113830.471468,2
4,0.215732,0.052847,0.005981,0.002046,10.000,{'model__alpha': 10},-36295.674895,-35623.825055,-145462.768753,-74976.284261,-39352.105970,-35239.653832,-34299.035463,-33786.752794,-69811.433298,-43507.535927,-54835.507025,33425.883327,1


In [8]:
ranked_results = results.sort_values(by='mean_test_score', ascending=[False])
ranked_results

,mean_fit_time,params,mean_test_score,dataset_name,use_pca,degree,rmse,nrmse
4,0.016136,{'model__alpha': 10},-3.344372e+04,numerical,True,1,33444,18.5%
34,0.009297,{'model__alpha': 10},-3.344372e+04,complete,True,1,33444,18.5%
33,0.009672,{'model__alpha': 1},-3.345710e+04,complete,True,1,33457,18.5%
3,0.015021,{'model__alpha': 1},-3.345710e+04,numerical,True,1,33457,18.5%
2,0.014935,{'model__alpha': 0.1},-3.345851e+04,numerical,True,1,33459,18.5%
32,0.008601,{'model__alpha': 0.1},-3.345851e+04,complete,True,1,33459,18.5%
31,0.009641,{'model__alpha': 0.01},-3.345865e+04,complete,True,1,33459,18.5%
1,0.014577,{'model__alpha': 0.01},-3.345865e+04,numerical,True,1,33459,18.5%
0,0.018591,{'model__alpha': 0.001},-3.345866e+04,numerical,True,1,33459,18.5%
30,0.009592,{'model__alpha': 0.001},-3.345866e+04,complete,True,1,33459,18.5%
